In [1]:
import sys
#sys.path.append('C:/Users/vodou/miniconda3/envs/aml/Lib/site-packages')

import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss, Dropout, Softmax, Linear, Conv2d, LayerNorm
import torch.optim as optim
import torch.nn.functional as F

import os
from PIL import Image
from torchvision import transforms

from torchvision.utils import make_grid
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, Dataset, TensorDataset

import math
import functools
import numpy as np
from tqdm import tqdm

from einops import rearrange

In [2]:
#Not necessary to below notebook
sys.path.append('C:/Users/vodou/Documents/Python Scripts')
from pushover_notifier import PushoverNotifier

notifier = PushoverNotifier(app_name="Task 3 (AML)")
notifier.redirect_print_to_pushover()

In [3]:
class CNNEncoder(nn.Module):

    def __init__(self, input_dim, output_dim=512, hidden_dims=[32, 64, 128, 256, 512, 1024]):

        super(CNNEncoder, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=input_dim, out_channels=hidden_dims[0], kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=hidden_dims[0], out_channels=hidden_dims[0], kernel_size=3, stride=1, padding=1)

        self.conv3 = nn.Conv2d(in_channels=hidden_dims[0], out_channels=hidden_dims[1], kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=hidden_dims[1], out_channels=hidden_dims[1], kernel_size=3, stride=1, padding=1)

        self.conv5 = nn.Conv2d(in_channels=hidden_dims[1], out_channels=hidden_dims[2], kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(in_channels=hidden_dims[2], out_channels=hidden_dims[2], kernel_size=3, stride=1, padding=1)

        self.conv7 = nn.Conv2d(in_channels=hidden_dims[2], out_channels=hidden_dims[3], kernel_size=3, stride=1, padding=1)
        self.conv8 = nn.Conv2d(in_channels=hidden_dims[3], out_channels=hidden_dims[3], kernel_size=3, stride=1, padding=1)

        self.conv9 = nn.Conv2d(in_channels=hidden_dims[3], out_channels=hidden_dims[4], kernel_size=3, stride=1, padding=1)
        self.conv10 = nn.Conv2d(in_channels=hidden_dims[4], out_channels=hidden_dims[4], kernel_size=3, stride=1, padding=1)

        self.conv11 = nn.Conv2d(in_channels=hidden_dims[4], out_channels=output_dim, kernel_size=3, stride=1, padding=1)
        self.conv12 = nn.Conv2d(in_channels=output_dim, out_channels=output_dim, kernel_size=3, stride=1, padding=1)

    def forward(self, x):

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))

        r1 = x

        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))

        r2 = x

        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))

        r3 = x

        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = F.relu(self.conv7(x))
        x = F.relu(self.conv8(x))

        r4 = x

        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = F.relu(self.conv9(x))
        x = F.relu(self.conv10(x))

        r5 = x

        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))

        return x, r1, r2, r3, r4, r5

In [4]:
class CNNDecoder(nn.Module):

    def __init__(self, output_dim=1, input_dim=512, hidden_dims=[512, 256, 128, 64, 32]):

        super(CNNDecoder, self).__init__()

        self.convt1 = nn.ConvTranspose2d(in_channels=input_dim, out_channels=hidden_dims[0], kernel_size=2, stride=2, padding=0)

        self.conv1 = nn.Conv2d(in_channels=hidden_dims[0]*2, out_channels=hidden_dims[0], kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=hidden_dims[0], out_channels=hidden_dims[0], kernel_size=3, stride=1, padding=1)

        self.convt2 = nn.ConvTranspose2d(in_channels=hidden_dims[0], out_channels=hidden_dims[1], kernel_size=2, stride=2, padding=0)

        self.conv3 = nn.Conv2d(in_channels=hidden_dims[1]*2, out_channels=hidden_dims[1], kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=hidden_dims[1], out_channels=hidden_dims[1], kernel_size=3, stride=1, padding=1)

        self.convt3 = nn.ConvTranspose2d(in_channels=hidden_dims[1], out_channels=hidden_dims[2], kernel_size=2, stride=2, padding=0)

        self.conv5 = nn.Conv2d(in_channels=hidden_dims[2]*2, out_channels=hidden_dims[2], kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(in_channels=hidden_dims[2], out_channels=hidden_dims[2], kernel_size=3, stride=1, padding=1)

        self.convt4 = nn.ConvTranspose2d(in_channels=hidden_dims[2], out_channels=hidden_dims[3], kernel_size=2, stride=2, padding=0)

        self.conv7 = nn.Conv2d(in_channels=hidden_dims[3]*2, out_channels=hidden_dims[3], kernel_size=3, stride=1, padding=1)
        self.conv8 = nn.Conv2d(in_channels=hidden_dims[3], out_channels=hidden_dims[3], kernel_size=3, stride=1, padding=1)

        self.convt5 = nn.ConvTranspose2d(in_channels=hidden_dims[3], out_channels=hidden_dims[4], kernel_size=2, stride=2, padding=0)

        self.conv9 = nn.Conv2d(in_channels=hidden_dims[4]*2, out_channels=hidden_dims[4], kernel_size=3, stride=1, padding=1)
        self.conv10 = nn.Conv2d(in_channels=hidden_dims[4], out_channels=hidden_dims[4], kernel_size=3, stride=1, padding=1)

        self.conv11 = nn.Conv2d(in_channels=hidden_dims[4], out_channels=output_dim, kernel_size=1, stride=1, padding=0)

    def forward(self, x, r1, r2, r3, r4, r5):

        x = self.convt1(x)
        x = torch.cat((r5, x), dim=1)

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))

        m1 = x

        x = self.convt2(x)
        x = torch.cat((r4, x), dim=1)

        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))

        m2 = x

        x = self.convt3(x)
        x = torch.cat((r3, x), dim=1)

        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))

        m3 = x

        x = self.convt4(x)
        x = torch.cat((r2, x), dim=1)

        x = F.relu(self.conv7(x))
        x = F.relu(self.conv8(x))

        m4 = x

        x = self.convt5(x)
        x = torch.cat((r1, x), dim=1)

        x = F.relu(self.conv9(x))
        x = F.relu(self.conv10(x))

        x = self.conv11(x)

        return x, m1, m2, m3, m4

In [5]:
class UNet(nn.Module):

    def __init__(self, input_dim=1):

        super(UNet, self).__init__()

        self.cnn_encoder = CNNEncoder(input_dim=input_dim)
        self.cnn_decoder = CNNDecoder()

        #self.transformer_encoder = TransformerEncoder(embed_dim=512, hidden_dim=512)

    def forward(self, x):

        x, r1, r2, r3, r4, r5 = self.cnn_encoder(x)
        #x = self.transformer_encoder(x)
        x, m1, m2, m3, m4 = self.cnn_decoder(x, r1, r2, r3, r4, r5)

        return x

In [6]:
unet = UNet()

In [7]:
class TensorSegmentationDataset(Dataset):
    def __init__(self, images, masks, transform=None):

        self.images = images
        self.masks = masks
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        mask = self.masks[idx]

        if self.transform:
            # Apply the transform to both the image and mask
            transformed = self.transform(image=image, mask=mask)
            image = transformed["image"]
            mask = transformed["mask"]

        return image, mask

In [8]:
class BCEWithDiceLoss(nn.Module):

    def __init__(self, pos_weight=None, size_penalty=0.01):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        self.dice = DiceLoss()
        self.size_penalty = size_penalty

    def forward(self, outputs, targets):
        bce_loss = self.bce(outputs, targets)
        dice_loss = self.dice(outputs, targets)
        size_penalty = self.size_penalty * outputs.sigmoid().sum(dim=(1, 2, 3)).mean()
        return bce_loss + dice_loss #+ size_penalty

In [9]:
class DiceLoss(nn.Module):

    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)  # Convert logits to probabilities
        intersection = (inputs * targets).sum(dim=(2, 3))  # Overlap
        dice = (2. * intersection + smooth) / (inputs.sum(dim=(2, 3)) + targets.sum(dim=(2, 3)) + smooth)
        return 1 - dice.mean()

In [10]:
def calculate_pos_weight(masks):
    total_pixels = torch.numel(masks)
    positive_pixels = (masks == 1).sum().item()
    negative_pixels = (masks == 0).sum().item()
    pos_weight = negative_pixels / (positive_pixels + 1e-6)  # Avoid division by zero
    return torch.tensor(pos_weight, dtype=torch.float)

In [11]:
def train_model(model, dataloader, optimizer='adam', loss_fn=nn.BCEWithLogitsLoss(), device='cpu', num_epochs=20):

    if optimizer == 'adam':
        optimizer=optim.Adam(model.parameters(), lr=0.0001)

    model = model.to(device)

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        epoch_loss = 0

        for images, masks in dataloader:
            images = images.to(device)  # Move images to GPU/CPU
            masks = masks.to(device)   # Move masks to GPU/CPU

            # For BCEWithLogitsLoss, masks must be float (not long/int)
            masks = masks.float()# if isinstance(loss_fn, nn.BCEWithLogitsLoss) else masks.long()

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)  # Predicted masks
            loss = loss_fn(outputs, masks)  # Compute the loss

            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss/len(dataloader):.4f}")

    return model


In [12]:
def test_model(model, X_test):

    model.eval()

    with torch.no_grad():

        predictions = model(X_test)

        # Apply sigmoid to convert logits to probabilities (for binary segmentation)
        predictions = torch.sigmoid(predictions)

        # Convert to binary mask with a threshold of 0.5 (for binary classification)
        predicted_masks = (predictions > 0.5).float()

    return predicted_masks

In [13]:
def iou(pred_mask, gt_mask):
    intersection = torch.sum(pred_mask * gt_mask)
    union = torch.sum(pred_mask) + torch.sum(gt_mask) - intersection
    return intersection / union if union != 0 else torch.tensor(0.0)

In [14]:
import numpy as np
import cv2
from sklearn.preprocessing import MinMaxScaler
import importlib
from skimage import exposure
from skimage.transform import rotate
import sys
import matplotlib.pyplot as plt

In [16]:
sys.path.append('C:/Users/vodou/Documents/Python Scripts')

import prog_tools as pt
import computer_interface as ci
import elastic

import task3 as helper


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\vodou\miniconda3\envs\aml\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\vodou\miniconda3\envs\aml\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\vodou\miniconda3\envs\aml\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "C:\Users

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

In [ ]:
train_data = helper.load_zipped_pickle("train.pkl")
test_data = helper.load_zipped_pickle("test.pkl")

In [ ]:
# remove all amateur data
train_data = [item for item in train_data if item.get('dataset') != 'amateur']

In [ ]:
train_names, train_videos, train_masks = helper.preprocess_train_data(train_data)
test_names, test_videos = helper.preprocess_test_data(test_data)

In [ ]:
prev = test_names[0]
nb = 1
nb_name = 0
name_and_seq = []
for name in test_names:
    if name != prev:
        name_and_seq.append((prev, nb_name))
        nb += 1
        nb_name = 0
        prev = name
    nb_name += 1

name_and_seq.append((prev, nb_name))
name_and_seq

In [ ]:
def extract_segmented_frames(video_frames, mask_frames):
    """
    Extracts video frames that have a segmented mitral valve labeled.

    Parameters:
    - video_frames: List of 3D numpy arrays (H x W x 1) representing video frames.
    - mask_frames: List of 3D numpy arrays (H x W x 1) representing segmentation masks.

    Returns:
    - segmented_frames: List of video frames where the mitral valve is labeled.
    - corresponding_masks: List of masks for the extracted frames.
    """
    segmented_frames = []
    corresponding_masks = []

    for video_frame, mask_frame in zip(video_frames, mask_frames):
        # Check if the mask contains any non-zero values (segmented region exists)
        if np.any(mask_frame):
            segmented_frames.append(video_frame)
            corresponding_masks.append(mask_frame)

    return segmented_frames, corresponding_masks

In [ ]:
train_frames, train_masks = extract_segmented_frames(train_videos, train_masks)

In [ ]:
target_size = (256, 256)

In [ ]:
resized_train_frames = [cv2.resize(frame, target_size, interpolation=cv2.INTER_NEAREST) for frame in train_frames]
resized_test_frames = [cv2.resize(frame, target_size, interpolation=cv2.INTER_NEAREST) for frame in test_videos]

In [ ]:
resized_train_masks = []

# mask frames are in int32, which isn't compatible with the cv2.resize function, so we need to convert them to uint8 first
for i, mask in enumerate(train_masks):

    if mask.dtype != np.uint8:
        mask = mask.astype(np.uint8)

    resized_mask = cv2.resize(mask, target_size, interpolation=cv2.INTER_NEAREST)
    resized_train_masks.append(resized_mask)

In [ ]:
resized_train_frames = np.expand_dims(resized_train_frames, axis=-1)
resized_train_masks = np.expand_dims(resized_train_masks, axis=-1)

In [ ]:
equalised_train = []
equalised_test = []

for i in range(len(resized_train_frames)):
    temp_array = []
    for j in range(len(resized_train_frames[i])):
        temp_array.append(exposure.equalize_hist(resized_train_frames[i][j]))
    equalised_train.append(temp_array)

for i in range(len(resized_test_frames)):
    equalised_test.append(exposure.equalize_hist(resized_test_frames[i]))

In [ ]:
normalised_train = []
normalised_test = []

scaler = MinMaxScaler(feature_range=(0, 1))

for i in range(len(equalised_train)):
    temp_array = []
    for j in range(len(equalised_train[i])):
        temp_array.append(scaler.fit_transform(equalised_train[i][j]))
    normalised_train.append(temp_array)

for i in range(len(equalised_test)):
    normalised_test.append(scaler.fit_transform(equalised_test[i]))

In [ ]:
augmented_train = []
train_masks_augmented = []

for i in range(len(normalised_train)):

    # Convert images and masks to numpy arrays
    frame = np.array(normalised_train[i])
    mask = np.array(resized_train_masks[i])

    # Elastic deformation
    deformed_frame, deformed_mask = elastic.deform_random_grid([frame, mask])
    augmented_train.append(deformed_frame)
    augmented_train.append(frame)
    train_masks_augmented.append(deformed_mask)
    train_masks_augmented.append(mask)

    # Rotation (example: 90 degrees)
    rotated_frame = rotate(frame, angle=90, mode='wrap')  # Wrap mode handles edges
    rotated_mask = rotate(mask, angle=90, mode='wrap')
    augmented_train.append(rotated_frame)
    train_masks_augmented.append(rotated_mask)

    # Horizontal Flip
    flipped_frame_h = np.fliplr(frame)  # Horizontal flip
    flipped_mask_h = np.fliplr(mask)
    augmented_train.append(flipped_frame_h)
    train_masks_augmented.append(flipped_mask_h)

    # Vertical Flip
    flipped_frame_v = np.flipud(frame)  # Vertical flip
    flipped_mask_v = np.flipud(mask)
    augmented_train.append(flipped_frame_v)
    train_masks_augmented.append(flipped_mask_v)

    # Combination of Flip and Rotation (e.g., flipped + rotated)
    flipped_rotated_frame = rotate(np.fliplr(frame), angle=90, mode='wrap')
    flipped_rotated_mask = rotate(np.fliplr(mask), angle=90, mode='wrap')
    augmented_train.append(flipped_rotated_frame)
    train_masks_augmented.append(flipped_rotated_mask)

In [ ]:
X_train = torch.from_numpy(np.array(augmented_train))
y_train = torch.from_numpy(np.array(train_masks_augmented))

X_test = torch.from_numpy(np.array(normalised_test))
X_test = X_test.unsqueeze(1)

In [ ]:
# Change the dimension order to fit the pytorch format:
X_train = X_train.permute(0, 3, 1, 2)
y_train = y_train.permute(0, 3, 1, 2)

In [ ]:
y_train.shape

In [ ]:
y_temp = y_train.squeeze(1)
y_temp.shape

In [ ]:
print('Beginning Training')

dataset = TensorSegmentationDataset(X_train, y_train)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)

pos_weight = calculate_pos_weight(y_temp)
del y_temp

#loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
loss_fn = BCEWithDiceLoss(pos_weight=pos_weight)

trained_unet = train_model(unet, dataloader, loss_fn=loss_fn, num_epochs=30)

print('Training Completed')

In [ ]:
torch.save(trained_unet.state_dict(), "trained_unet.pth")

In [ ]:
#trained_unet = unet
#trained_unet.load_state_dict(torch.load("trained_unet.pth"))

In [ ]:
def predict_in_batches(model, dataloader, device='cpu', threshold=0.5):
    model = model.to(device)  # Move model to device
    model.eval()  # Set model to evaluation mode

    all_predictions = []

    with torch.no_grad():  # Disable gradient computation for inference
        for batch in dataloader:
            # Unpack the batch (assuming no targets)
            inputs = batch[0].to(device)  # Only take the input tensor

            # Predict
            batch_predictions = model(inputs)

            # Apply thresholding to convert to binary masks
            binary_predictions = (batch_predictions >= threshold).float()  # Convert to binary (0 or 1)

            # Store predictions
            all_predictions.append(binary_predictions.cpu())  # Move to CPU for storage

    # Concatenate all batch predictions into a single tensor
    return torch.cat(all_predictions, dim=0)

In [ ]:
test_dataset = TensorDataset(X_test)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)

In [ ]:
binary_masks = predict_in_batches(trained_unet, test_dataloader, threshold=0.7)
print('Predictions Calculated')

In [ ]:
def mask_left_side(tensor, cutoff_column):
    tensor[:, :, :cutoff_column] = 0
    return tensor

In [ ]:
height, width = 256, 256
cutoff_column = width // 2

masked_masks = [mask_left_side(mask.clone(), cutoff_column) for mask in binary_masks]

In [ ]:
binary_masks = masked_masks

In [ ]:
np.save("binary_masks.npy", binary_masks)

binary_masks = np.load("binary_masks.npy")

In [ ]:
binary_masks = torch.from_numpy(np.array(binary_masks))
binary_masks = binary_masks.permute(0, 2, 3, 1)
binary_masks = np.array(binary_masks)
binary_masks.shape

In [ ]:
def quantize_image(tensor, levels=4, min_val=0, max_val=255):

    # Find the min and max values of the tensor
    min_val, max_val = tensor.min(), tensor.max()

    # Compute the step size for quantization
    step_size = (max_val - min_val) / (levels - 1)

    quantized_tensor = torch.round((tensor - min_val) / step_size) * step_size + min_val

    return quantized_tensor

In [ ]:
X_test_quantized = quantize_image(X_test, 4)

In [ ]:
temp = torch.from_numpy(binary_masks)
temp = temp.permute(0, 3, 1, 2)

In [ ]:
for i in range(len(temp)):
    temp_mask = (X_test[i] >= 0.6)
    temp_tensor = temp[i]
    temp[i] = temp_tensor * temp_mask

In [ ]:
for i in range(3):
    #image_tensor = X_test_quantized[i]
    image_tensor = X_test[i]
    #mask_tensor = binary_masks[i]
    mask_tensor = temp[i]

    # Prepare the image for display
    image_np = image_tensor.permute(1, 2, 0).numpy()  # Convert to (H, W, C)

    # Prepare the mask for overlay (ensure it's a NumPy array)
    mask_np = mask_tensor.squeeze(0).numpy()

    # Plot the image and overlay the mask
    plt.figure(figsize=(8, 8))
    plt.imshow(image_np)  # Display the image
    plt.imshow(mask_np, cmap="jet", alpha=0.5)  # Overlay the mask with transparency
    plt.axis('off')  # Hide axes
    plt.show()

In [ ]:
binary_masks = temp

In [ ]:
print(binary_masks.shape)
binary_masks = binary_masks.permute(0, 2, 3, 1)
binary_masks = np.array(binary_masks)
binary_masks.shape

In [ ]:
resized_binary_mask = []

for i in range(len(binary_masks)):
    current_entry = cv2.resize(binary_masks[i], test_videos[i].shape[:2][::-1], interpolation=cv2.INTER_NEAREST)
    #temp = np.array(current_entry)
    #temp2 = np.array(test_videos[i])
    resized_binary_mask.append(current_entry)

del binary_masks

In [ ]:
recreated_test_videos = []

current_index = 0

for i in range(len(name_and_seq)):
    name = name_and_seq[i][0]
    seq = name_and_seq[i][1]
    temp = np.array(resized_binary_mask[current_index:current_index+seq])
    current_index += seq
    temp = temp.transpose(1, 2, 0)
    recreated_test_videos.append(temp)

In [ ]:
for i in range(789,794):
    image_tensor = torch.from_numpy(np.array(test_videos[i]))
    mask_tensor = torch.from_numpy(np.array(resized_binary_mask[i]))  # Binary segmentation mask (0s and 1s)
    print(mask_tensor.shape)
    print(image_tensor.shape)

    # Prepare the image for display
    image_np = image_tensor#.permute(1, 2, 0).numpy()  # Convert to (H, W, C)

    # Prepare the mask for overlay (ensure it's a NumPy array)
    mask_np = mask_tensor#.squeeze(0).numpy()

    # Plot the image and overlay the mask
    plt.figure(figsize=(8, 8))
    plt.imshow(image_np)  # Display the image
    plt.imshow(mask_np, cmap="jet", alpha=0.5)  # Overlay the mask with transparency
    plt.axis('off')  # Hide axes
    plt.show()

In [ ]:
#del trained_unet
del resized_binary_mask

In [ ]:
flattened_masks = []

for item in recreated_test_videos:
    flattened = item.flatten()
    flattened_masks.append(flattened)

In [ ]:
def get_sequences(arr):
    first_indices, last_indices, lengths = [], [], []
    n, i = len(arr), 0
    arr = [0] + list(arr) + [0]
    for index, value in enumerate(arr[:-1]):
        if arr[index+1]-arr[index] == 1:
            first_indices.append(index)
        if arr[index+1]-arr[index] == -1:
            last_indices.append(index)
    lengths = list(np.array(last_indices)-np.array(first_indices))
    return first_indices, lengths

In [ ]:
index_length = []

for array in flattened_masks:
    first_indices, lengths = get_sequences(array)
    index_length.append((first_indices, lengths))

print('index_length sequences calculated')

In [ ]:
with open("index_length.txt", "w") as file:
    for item in index_length:
        file.write(f"{item}\n")

In [ ]:
submission = []

for video_mask, (video_name, _) in zip(index_length, name_and_seq):
    indices = video_mask[0]
    lengths = video_mask[1]
    for i in range(len(video_mask[0])):
        pair = [indices[i], lengths[i]]
        name = video_name + "_" + str(i)
        submission.append((name, pair))

In [ ]:
submission

In [ ]:
y_pred_to_save = []
ids_to_save = []

for name, info in submission:
    ids_to_save.append(name)
    y_pred_to_save.append(info)

In [ ]:
import pandas as pd

df = pd.DataFrame({"id":ids_to_save, "value":[list(map(int, minili)) for minili in y_pred_to_save]})
df.to_csv(f"mysubmissionfile.csv", index=False)